In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [ ]:
##Loading the titanic dataset
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
#Exploring the data
print(dftrain.head())
print(dftrain.describe())
dftrain.shape
#Exploring dataset.
#dftrain.age.hist(bins=20)
#dftrain.sex.value_counts().plot(kind="barh")
#dftrain['class'].value_counts().plot(kind="barh")
#pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

In [ ]:
#Creating feature columns
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

for feature_column in feature_columns:
    print(feature_column)

In [ ]:
#Making the input function
def make_input_fn(data_df, label_df, num_epochs=30, shuffle=True, batch_size=16):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [ ]:
#Inspect the dataset using the input function
ds = make_input_fn(dftrain, y_train, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['sex'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

age_column = feature_columns[7]
print(tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy())

gender_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(gender_column)])(feature_batch).numpy()

In [ ]:
#Training the model and printing the accuracry
age_x_gender = tf.feature_column.crossed_column(['age', 'sex'], hash_bucket_size=100)
fare_x_gender = tf.feature_column.crossed_column(['fare', 'sex'], hash_bucket_size=100)
class_x_gender = tf.feature_column.crossed_column(['class', 'sex'], hash_bucket_size=100)
deck_x_gender = tf.feature_column.crossed_column(['deck', 'sex'], hash_bucket_size=100)


derived_feature_columns = [age_x_gender, fare_x_gender, class_x_gender, deck_x_gender]

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns+derived_feature_columns)

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])

In [ ]:
pred_dicts = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[4])
print(y_eval.loc[4])
print(pred_dicts[4]['probabilities'][1])
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)